# DEEPWALK IMPLEMENTATION

In [2]:
PATH = '/Users/silviaarellanogarcia/Documents/MSc MACHINE LEARNING/Advanced Machine Learning/Project/Datasets/Flickr-dataset'
SAVE_PATH = '/Users/silviaarellanogarcia/Documents/MSc MACHINE LEARNING/Advanced Machine Learning/Project/embeddings_deepwalk'

In [1]:
!pip install umap-learn
!pip install networkx
!pip install gensim
!pip install seaborn matplotlib pandas
!pip install scikit-learn
!pip install scikit-multilearn
!pip install torch-geometric
!pip install torch

  Using cached umap_learn-0.5.5-py3-none-any.whl
  Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached scipy-1.10.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.5 MB)
  Using cached scikit_learn-1.3.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached numba-0.58.1-cp38-cp38-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.7 kB)
  Using cached pynndescent-0.5.11-py3-none-any.whl.metadata (6.8 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached llvmlite-0.41.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.8 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl.metadata (10.0 kB)
Using cached numba-0.58.1-cp38-cp38-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.7 MB)
Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.w

In [22]:
import networkx as nx
import random
from gensim.models import Word2Vec # Needed for Word2Vec
import numpy as np

import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer

import numpy as np
from sklearn.model_selection import KFold
from sklearn.multiclass import OneVsRestClassifier

from multiprocessing import cpu_count
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import CitationFull
from torch_geometric.datasets import HeterophilousGraphDataset
from torch_geometric.datasets import Reddit
from torch_geometric.datasets import PPI

from torch_geometric.utils import to_networkx
from gensim.models.keyedvectors import KeyedVectors
import torch

from concurrent.futures import ProcessPoolExecutor
from six import string_types
from gensim.models.word2vec import Vocab

import scipy.io as sio
from scipy import sparse

from collections import defaultdict
from scipy.sparse import coo_matrix
from sklearn.utils import shuffle as skshuffle

In [13]:
# HYPERPARAMETERS
window_size = 10 ## 10 --> Suggested in the Khosla comparative paper
embedding_size = 128
walks_per_vertex = 80 ##80 --> Suggested in the Khosla comparative paper
walk_length = 40 #40 --> Suggested in the Khosla comparative paper

### IMPLEMENTATION OF THE DEEPWALK METHOD.

In [4]:
def Random_Walk(G, vi, t):
  '''
  Inputs:
    G: Graph
    vi: initial vertex of the random walk
    t: walk length (the walks could have different length according to the paper)
  Output:
    Wvi: sequence of vertices visited in the random walk (starting from vi).
  '''
  Wvi = []
  Wvi.append(str(vi)) # The initial vertex is always visited
  last_visited = vi

  for i in range(t):
    neighbors_last_vi = list(G.neighbors(last_visited))
    last_visited = random.choice(neighbors_last_vi)
    Wvi.append(str(last_visited))

  return Wvi

In [5]:
# Optimization method
class Skipgram(Word2Vec):
    """A subclass to allow more customization of the Word2Vec internals."""

    def __init__(self, **kwargs):

        self.vocabulary_counts = None

        kwargs["min_count"] = kwargs.get("min_count", 0)
        kwargs["workers"] = kwargs.get("workers", cpu_count())
        kwargs["vector_size"] = 128
        kwargs["sentences"] = kwargs.get("sentences", None)
        kwargs["window"] = kwargs.get("window", 10)
        kwargs["sg"] = 1
        kwargs["hs"] = 1

        super(Skipgram, self).__init__(**kwargs)

In [6]:
def DeepWalk(G, w, d, gamma, t):
  '''
  Inputs:
    G: Graph with vertices V and edges E
    w: window size
    d: embedding size
    gamma: walks per vertex
    t: walk length
  Outputs:
    phi: matrix of vertex representations
  '''
  vertices_in_G = list(G.nodes)
  Wvi = [] # Array where each element is a list of arrays containing the walks that start from that vertex.

  for i in range(0, gamma):
    random.shuffle(vertices_in_G) # This variable receives the name O in the paper
    for vi in vertices_in_G:
      Wvi.append(Random_Walk(G, vi, t))

  print("Finish random walk")
  
  phi_model = Skipgram(sentences=Wvi, window=w, min_count=0, trim_rule=None)

  return phi_model

In [7]:
# Subroutine to label a node with MAX-VOTE --> NOT NECESSARY. The original paper uses this as a baseline
def Max_Vote(G, node, k, L):
  '''
  node: node to label --> v in the paper
  neighbors: neighbors of the node to label --> N(v) in the paper
  k: Number of labels that will be assigned to the vertex
  L: total number of possible labels

  Output:
    k_lab: most frequent k labels
  '''
  freq_labels = np.zeros(L)
  k_lab = np.full(k , -1) # Set to -1 all the chosen labels
  neighbors = list(G.neighbors(node))

  for n in neighbors:
    labels_neighbor = G.nodes[n]['group'] # Is it necessary to check if the neighbor is empty, or it won't have any effect?
    print(labels_neighbor)
    for l in labels_neighbor:
      freq_labels[l] += 1

  used_labels = np.count_nonzero(freq_labels)
  for i in range(min(k, used_labels)):
    k_lab[i] = np.argmax(freq_labels)
    freq_labels[k_lab[i]] = -1 # We mark it as used, but at the same time we differentiate this label and the ones that haven't appeared.

  if(k > used_labels): # In case we have to assign more classes than the ones of our neighbors, we choose randomly
    zero_indices = np.where(freq_labels == 0)[0]
    random_zero_indices = np.random.choice(zero_indices, size=(k - used_labels), replace=False)
    k_lab[used_labels:k] = random_zero_indices

  return k_lab

In [8]:
# Some methods need to have all the labels inside an array. Depending on the case, that array has to be composed with arrays/sets containing the labels assigned to each node

def get_set_labels(G):
    '''
    Input:
        G: Graph
    Output:
        labels: List of sets containing labels. Each set corresponds to the groups of id = index + 1
    '''

    labels = []

    for n in G.nodes:
        l = set(G.nodes[n].get('group_belonging', []))  # Ensure an empty list is converted to an empty set
        labels.append(l)

    return labels

def get_array_labels(G):
  '''
  Input:
    G: Graph
  Output:
    labels: Array with labels. Each position of the array will correspond to the groups of id = index + 1
  '''

  labels = []

  for n in G.nodes:
      l = G.nodes[n].get('group_belonging')
      labels.append(l)
  return labels

### Running DeepWalk in the YouTube Dataset

In [9]:
# dataset = CitationFull(root='/tmp/Cora', name='Cora') ## CORA
# dataset = Reddit("./") ## REDDIT
# dataset = HeterophilousGraphDataset(root="./", name='amazon_ratings') ## AMAZON RATINGS
dataset = Planetoid(root='/tmp/PubMed', name='PubMed') ## PUBMED
data = dataset[0]
G_YT = to_networkx(data, to_undirected=False)

Processing...
Done!


In [10]:
# Access node labels
labels = data.y

# Convert node labels to a dictionary with node indices as keys
group_dict = {i: [labels[i].item()] for i in range(len(labels))}

# Add group labels to the nodes
for user_id, groups in group_dict.items():
    nx.set_node_attributes(G_YT, {user_id: groups}, 'group_belonging')

print("Number of nodes:", G_YT.number_of_nodes())
print("Number of edges:", G_YT.number_of_edges())

Number of nodes: 19717
Number of edges: 88648


In [31]:
## Obtention of a .mat file with the edge list

# Obtention of edgelist
edgelist = nx.generate_edgelist(G_YT, data=False)
#nx.write_edgelist(G_YT, "pubmed.edgelist", data=False)

# Create a label matrix
num_nodes = len(G_YT.nodes)
num_classes = len(set(map(lambda x: x[0], list(group_dict.values()))))
label_matrix = np.zeros((num_nodes, num_classes))

for node, labels in group_dict.items():
    for label in labels:
        # Subtract 1 from label to adjust for zero-based indexing
        label_matrix[node - 1, label - 1] = 1

Adj = nx.adjacency_matrix(G_YT)
label_matrix_sparse = sparse.csr_matrix(label_matrix)
sio.savemat('/Users/silviaarellanogarcia/Documents/MSc MACHINE LEARNING/Advanced Machine Learning/Project/embeddings_deepwalk/pubmed2.mat', {'network': Adj, 'group': label_matrix_sparse})

In [14]:
# Graph: G_YT
phi_model_YT = DeepWalk(G_YT, window_size, embedding_size, walks_per_vertex, walk_length) # Phi represents the learned embedding matrix.

Finish random walk


In [60]:
# Save the model in Word2Vec format
phi_model_YT.wv.save_word2vec_format(SAVE_PATH + '/model_pubmed_directed_80_40.embedding')

In [15]:
# Get the embedding vectors
phi_vectors_YT = phi_model_YT.wv.vectors
print(phi_vectors_YT)

[[ 0.2742879   0.16498137  0.21245699 ...  0.13313136  0.07546434
  -0.03327994]
 [-0.38610804  0.1626573   0.14413613 ... -0.21662538 -0.09398244
  -0.03844263]
 [-0.19908889 -0.5658283  -0.38879186 ...  0.37729552 -0.30201608
  -0.10404451]
 ...
 [ 0.5277188  -0.29266483 -0.3273233  ...  0.37128177 -0.18906634
   0.44112396]
 [-0.12426006 -0.39650354  0.11178714 ... -0.37924275 -0.45998713
   0.47173586]
 [ 0.29561248 -0.8507624  -0.06788737 ... -0.4310972  -0.7873194
  -0.09269765]]


In [63]:
# Save the vectors in a numpy file
np.save(SAVE_PATH + '/vectors_model_pubmed_directed_80_40.npy', phi_vectors_YT)

In [ ]:
# LOAD PHI VECTORS YT FROM FILE
phi_vectors_YT = np.load(SAVE_PATH + '/vectors_model_cora_directed_80_40.npy')

In [29]:
# LOAD PHI MODEL
phi_model_YT = KeyedVectors.load_word2vec_format(SAVE_PATH + '/model_cora_80_40.embedding')

In [30]:
# Visualize the loaded vectors
phi_model_YT.vectors

array([[ 4.45148319e-01, -8.30312744e-02, -3.39485615e-01, ...,
        -8.87622610e-02,  2.34206527e-01,  1.55090705e-01],
       [ 2.82594532e-01,  2.03033134e-01,  1.44470811e-01, ...,
        -1.80515721e-01,  2.15088263e-01,  1.64272547e-01],
       [ 3.58948499e-01,  1.77176595e-01, -8.48050117e-02, ...,
        -4.79790121e-01, -9.95380506e-02,  1.48593456e-01],
       ...,
       [-3.99145260e-02,  7.22329691e-02, -8.29841495e-02, ...,
        -9.61652398e-03,  3.82785161e-04, -7.63317896e-03],
       [ 8.13602135e-02, -9.95194495e-01,  2.53794134e-01, ...,
        -3.24240237e-01,  2.48191029e-01, -1.90682977e-01],
       [ 5.05192317e-02, -1.01612605e-01, -1.49148166e-01, ...,
        -3.09588045e-01,  2.86159873e-01, -2.50462711e-01]], dtype=float32)

In [16]:
# To improve the access to the adjacency matrix, we pass the info to a dictionary, where all the nodes have a entry, and the values aare the neighbours
def adj_dictionary(x):
    G = defaultdict(lambda: set())
    cx = x.tocoo() # Return a COOrdinate representation of this matrix
    for i,j,v in zip(cx.row, cx.col, cx.data):
        G[i].add(j)
    return {str(k): [str(x) for x in v] for k, v in G.items()}

In [17]:
class TopKRanker(OneVsRestClassifier):
    def predict(self, X, top_k_list):
        assert X.shape[0] == len(top_k_list)
        probs = np.asarray(super(TopKRanker, self).predict_proba(X))
        all_labels = []
        for i, k in enumerate(top_k_list):
            probs_ = probs[i, :]
            labels = self.classes_[probs_.argsort()[-k:]].tolist()
            all_labels.append(labels)
        return all_labels

In [18]:
def get_array_str_labels(G):
  '''
  Input:
    G: Graph
  Output:
    labels: Array with labels. Each position of the array will correspond to the groups of id = index + 1
  '''

  labels = []

  for n in G.nodes:
      l = G.nodes[n].get('group_belonging')
      labels.append(str(l))
  return labels

In [19]:
# Method used when the dataset doesn't have all the nodes labeled and we need to evaluate

def filter_phi(phi, labels_set):
    '''
    Input:
      phi: Embedding matrix
      labels_array: Array containing the

    Output:
      phi_new: Phi Embedding matrix containing only the rows of the labeled nodes
    '''
    label_np = np.array(labels_set) # Convert the set to a NumPy array

    indices_containing_labels = [bool(node_set) for node_set in label_np] # List with True for those indices that correspond to nodes with labels, and False for those which don't have any

    # Filter nodes with labels
    filtered_labels = label_np[indices_containing_labels] # Numpy array containing the labels of those nodes that are labeled

    # Get the indices to keep in the original order
    indices_to_keep = np.where(indices_containing_labels)[0] # Id's of the indices (id_nodes - 1) that have a label

    # Filter the embedding matrix
    filtered_phi = phi[indices_to_keep, :]

    return filtered_phi, filtered_labels

In [20]:
labels_sets = get_set_labels(G_YT)
new_phi, new_labels = filter_phi(phi_vectors_YT, labels_sets)
features_matrix = np.asarray([phi_model_YT.wv[str(node)] for node in range(len(G_YT))])

ATTENTION!
Depending on the dataset, there are some lines that need to be commented/discommented. Check the comments in capital letters

In [23]:
adj_G_YT = nx.adjacency_matrix(G_YT, nodelist=None, dtype=None, weight='weight') # Adjacency matrix: Square matrix of N x N size used to represent the connections between the edges of a graph.
adj_graph = adj_dictionary(adj_G_YT)

num_labels = dataset.num_classes

#mlb = MultiLabelBinarizer(classes=range(1, num_labels + 1))
mlb = MultiLabelBinarizer(classes=range(num_labels)) ## FOR DATASETS THAT START WITH LABEL 0

labels_bin = mlb.fit_transform(new_labels)
labels_bin = np.array(labels_bin)

try:
    features_matrix = np.asarray([phi_model_YT.wv[str(node+1)] for node in range(len(G_YT))])
except:
    features_matrix = np.asarray([phi_model_YT.wv[str(node)] for node in range(len(G_YT))])

shuffles = []
for x in range(10): # Decide how many shuffles I want to make
    shuffles.append(skshuffle(features_matrix, labels_bin)) # The sklearn shuffle shuffles arrays or sparse matrices in a consistent way.


all_results = defaultdict(list)

training_percents = [0.1, 0.5, 0.9]

for train_percent in training_percents:
    for shuf in shuffles:
        X, y = shuf ## X corresponds to the phi_vectors and y to the labels.

        training_size = int(train_percent * len(X))

        X_train = X[:training_size, :]
        y_train_ = y[:training_size]

        y_train = [[] for x in range(len(y_train_))]

        y_train_sparse = coo_matrix(y_train_)
        cy =  y_train_sparse.tocoo()

        for i, j in zip(cy.row, cy.col):
            y_train[i].append(j)

        assert sum(len(l) for l in y_train) == np.count_nonzero(y_train_)

        X_test = X[training_size:, :]
        y_test_ = y[training_size:]

        y_test = [[] for _ in range(len(y_test_))]

        y_test_sparse = coo_matrix(y_test_)
        cy =  y_test_sparse.tocoo()
        for i, j in zip(cy.row, cy.col):
            y_test[i].append(j)

        #print(y_test)
        clf = TopKRanker(LogisticRegression()) # creates an instance of TopKRanker with a LogisticRegression model as the base classifier.
        clf.fit(X_train, y_train_)

        # find out how many labels should be predicted
        top_k_list = [len(l) for l in y_test]
        preds = clf.predict(X_test, top_k_list)

        results = {}
        averages = ["micro", "macro"]
        for average in averages:
            results[average] = f1_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), average=average)

        all_results[train_percent].append(results)


In [25]:
print ('Results, using embeddings of dimensionality', X.shape[1])
print ('-------------------')
for train_percent in sorted(all_results.keys()):
    print ('Train percent:', train_percent)
for index, result in enumerate(all_results[train_percent]):
    print ('Shuffle #%d:   ' % (index + 1), result)
avg_score = defaultdict(float)
for score_dict in all_results[train_percent]:
    for metric, score in score_dict.items():
        avg_score[metric] += score
for metric in avg_score:
    avg_score[metric] /= len(all_results[train_percent])
print ('Average score:', dict(avg_score))
print ('-------------------')

Results, using embeddings of dimensionality 128
-------------------
Train percent: 0.1
Train percent: 0.5
Train percent: 0.9
Shuffle #1:    {'micro': 0.8078093306288032, 'macro': 0.7978228157109367}
Shuffle #2:    {'micro': 0.8002028397565923, 'macro': 0.786765798692207}
Shuffle #3:    {'micro': 0.8199797160243407, 'macro': 0.8017392153172597}
Shuffle #4:    {'micro': 0.8017241379310345, 'macro': 0.7828446256535169}
Shuffle #5:    {'micro': 0.8022312373225152, 'macro': 0.7874813351806175}
Shuffle #6:    {'micro': 0.8057809330628803, 'macro': 0.7926314412931766}
Shuffle #7:    {'micro': 0.8179513184584178, 'macro': 0.8093564877769525}
Shuffle #8:    {'micro': 0.8042596348884381, 'macro': 0.7900774490273822}
Shuffle #9:    {'micro': 0.813894523326572, 'macro': 0.8034704351492418}
Shuffle #10:    {'micro': 0.8113590263691683, 'macro': 0.8026878410592477}
Average score: {'micro': 0.8085192697768763, 'macro': 0.7954877444860539}
-------------------
